In [2]:
import numpy as np
import pickle
import os

seed = 2023

In [ ]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-17 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-17 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-17 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-17 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-17 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# Baseline: FFN baseline

Using the embeddings for the sentences directly in a FFN to predict the client talk type.

In [7]:
num_epochs = 100
hidden_dim_sizes = [[64, 64], [128, 128], [256, 256]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [8]:
hidden_dim_sizes

[[64, 64], [128, 128], [256, 256]]

In [9]:
learning_rates

[0.001, 0.0001, 0.0005]

In [ ]:
kwargs = {
    "num_epochs": num_epochs,
    "x_data": sbert_embeddings[client_index],
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "hidden_dim_sizes": hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

In [20]:
ffn_current_kfold, best_ffn_current_kfold, _, __ = ffn_hyperparameter_search(
    results_output=f"{output_dir}/ffn_current_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_kfold_best_model.csv


In [21]:
ffn_current_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,model_id
0,None,0.639272,0.518299,"[0.7679810958499483, 0.4306732055283103, 0.356...",0.514406,"[0.7847872019317839, 0.4684772065955383, 0.289...",0.537404,"[0.7518796992481203, 0.39851485148514854, 0.46...",None,0.661047,...,"(64, 64)",0.5,0.0010,1,focal,2,True,5,64,0
0,None,0.645211,0.521731,"[0.7727807172251616, 0.43989314336598395, 0.35...",0.518379,"[0.7857142857142857, 0.47775628626692457, 0.29...",0.537770,"[0.760266049739734, 0.4075907590759076, 0.4454...",None,0.658799,...,"(64, 64)",0.5,0.0010,12,focal,2,True,5,64,0
0,None,0.629310,0.508992,"[0.761365330153525, 0.4254754533392304, 0.3401...",0.505292,"[0.7856044294063365, 0.45853193517635843, 0.27...",0.529996,"[0.7385772122614228, 0.39686468646864687, 0.45...",None,0.649807,...,"(64, 64)",0.5,0.0010,123,focal,2,True,5,64,0
0,None,0.643103,0.493964,"[0.7768313458262351, 0.39620081411126185, 0.30...",0.496532,"[0.7629670942554378, 0.43843843843843844, 0.28...",0.495107,"[0.7912087912087912, 0.3613861386138614, 0.332...",None,0.656230,...,"(64, 64)",0.5,0.0001,1,focal,2,True,5,64,1
0,None,0.646743,0.521010,"[0.7752165614447218, 0.43796255986068783, 0.34...",0.516301,"[0.787354607813898, 0.46359447004608295, 0.297...",0.534033,"[0.763447079236553, 0.415016501650165, 0.42363...",None,0.657354,...,"(64, 64)",0.5,0.0001,12,focal,2,True,5,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.636782,0.510545,"[0.7648449039881833, 0.4371900826446281, 0.329...",0.504634,"[0.7817028985507246, 0.4379139072847682, 0.294...",0.519904,"[0.7486986697513013, 0.43646864686468645, 0.37...",None,0.664419,...,"(256, 256)",0.1,0.0001,12,focal,2,True,5,64,25
0,None,0.639464,0.513886,"[0.7655882352941176, 0.43705857914416285, 0.33...",0.508348,"[0.7788749251944943, 0.4401673640167364, 0.306...",0.522247,"[0.7527472527472527, 0.43399339933993397, 0.38]",None,0.665543,...,"(256, 256)",0.1,0.0001,123,focal,2,True,5,64,25
0,None,0.634674,0.514114,"[0.7663690476190477, 0.42844522968197885, 0.34...",0.509890,"[0.7893930104230533, 0.4610266159695818, 0.279...",0.534938,"[0.7446500867553499, 0.40016501650165015, 0.46]",None,0.653179,...,"(256, 256)",0.1,0.0005,1,focal,2,True,5,64,26
0,None,0.640421,0.517552,"[0.7701826753093693, 0.42434936038817817, 0.35...",0.512638,"[0.784984984984985, 0.45592417061611373, 0.297...",0.534567,"[0.7559282822440717, 0.39686468646864687, 0.45...",None,0.658478,...,"(256, 256)",0.1,0.0005,12,focal,2,True,5,64,26


In [22]:
ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1276216/3418571787.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
(64, 64)   0.1          0.0001         0.635824  0.511660   0.505265   
                        0.0005         0.634930  0.512910   0.507614   
                        0.0010         0.630077  0.510390   0.504599   
           0.2          0.0001         0.637165  0.511612   0.505694   
                        0.0005         0.629374  0.512933   0.505438   
                        0.0010         0.626117  0.508063   0.501684   
           0.5          0.0001         0.645211  0.509257   0.507438   
                        0.0005         0.638825  0.515069   0.509710   
                        0.0010         0.637931  0.516340   0.512692   
(128, 128) 0.1          0.0001         0.637803  0.514196   0.507761   
                        0.0005         0.636782  0.511728   0.506048   
                        0.0010         0.635377  0.511005   0.507473   
           0.2          0.0001         0.637612  0.512890   0.506711   
                        0.0005         0.637037  0.516536   0.509918   
                        0.0010         0.637484  0.514514   0.510059   
           0.5          0.0001         0.640996  0.513882   0.508436   
                        0.0005         0.631034  0.509554   0.503991   
                        0.0010         0.630332  0.512102   0.506496   
(256, 256) 0.1          0.0001         0.637739  0.511845   0.506073   
                        0.0005         0.641699  0.517209   0.513440   
                        0.0010         0.631226  0.507740   0.502620   
           0.2          0.0001         0.639017  0.513584   0.507732   
                        0.0005         0.635185  0.514226   0.508284   
                        0.0010         0.636845  0.510194   0.505569   
           0.5          0.0001         0.639847  0.514001   0.508301   
                        0.0005         0.629374  0.510273   0.504064   
                        0.0010         0.643678  0.520936   0.516213   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
(64, 64)   0.1          0.0001         0.524039        0.658531  0.546615   
                        0.0005         0.529921        0.655855  0.546246   
                        0.0010         0.528383        0.654089  0.546322   
           0.2          0.0001         0.523203        0.658157  0.544536   
                        0.0005         0.531583        0.654624  0.548361   
                        0.0010         0.527690        0.652537  0.545799   
           0.5          0.0001         0.516472        0.657675  0.532637   
                        0.0005         0.528625        0.659067  0.547164   
                        0.0010         0.535057        0.656551  0.544456   
(128, 128) 0.1          0.0001         0.526100        0.660083  0.548817   
                        0.0005         0.524308        0.659762  0.545945   
                        0.0010         0.527885        0.655641  0.543299   
           0.2          0.0001         0.523813        0.660833  0.547724   
                        0.0005         0.532092        0.657729  0.548325   
                        0.0010         0.531021        0.657461  0.543954   
           0.5          0.0001         0.524230        0.660726  0.546286   
                        0.0005         0.527028        0.657568  0.549237   
                        0.0010         0.532172        0.652430  0.544334   
(256, 256) 0.1          0.0001         0.520938        0.665115  0.550463   
                        0.0005         0.533010        0.660726  0.547288   
                        0.0010         0.524194        0.656230  0.547520   
           0.2          0.0001         0.523050        0.663081  0.549187   
                        0.0005         0.531120        0.658424  0.547853   
                        0

In [23]:
best_ffn_current_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.636973,0.511104,"[0.7645756457564574, 0.43752594437525943, 0.33...",0.505237,"[0.7808260476334037, 0.4402673350041771, 0.294...",0.520663,"[0.7489878542510121, 0.43481848184818483, 0.37...",None,0.665382,...,"[0.7901234567901234, 0.469281045751634, 0.3978...","(256, 256)",0.1,0.0001,1,focal,2,True,5,64
0,None,0.636782,0.510545,"[0.7648449039881833, 0.4371900826446281, 0.329...",0.504634,"[0.7817028985507246, 0.4379139072847682, 0.294...",0.519904,"[0.7486986697513013, 0.43646864686468645, 0.37...",None,0.664419,...,"[0.7876039304610734, 0.46797385620915033, 0.40...","(256, 256)",0.1,0.0001,12,focal,2,True,5,64
0,None,0.639464,0.513886,"[0.7655882352941176, 0.43705857914416285, 0.33...",0.508348,"[0.7788749251944943, 0.4401673640167364, 0.306...",0.522247,"[0.7527472527472527, 0.43399339933993397, 0.38]",None,0.665543,...,"[0.7903754094230284, 0.4627450980392157, 0.411...","(256, 256)",0.1,0.0001,123,focal,2,True,5,64


In [24]:
best_ffn_current_kfold["f1"].mean()

0.5118449028528574

In [25]:
best_ffn_current_kfold["precision"].mean()

0.5060731871074536

In [26]:
best_ffn_current_kfold["recall"].mean()

0.5209379530588493

In [27]:
np.stack(best_ffn_current_kfold["f1_scores"]).mean(axis=0)

array([0.76500293, 0.4372582 , 0.33327358])

In [28]:
np.stack(best_ffn_current_kfold["precision_scores"]).mean(axis=0)

array([0.78046796, 0.43944954, 0.29830207])

In [29]:
np.stack(best_ffn_current_kfold["recall_scores"]).mean(axis=0)

array([0.75014459, 0.43509351, 0.37757576])